In [ ]:
import os
import re
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
from dotenv import load_dotenv

# ============================================
# ENHANCED MEDICAL CHATBOT QUERY SYSTEM
# ============================================

class MedicalChatbot:
    def __init__(self, index_name="medical-knowledge-v2", model_name='all-MiniLM-L6-v2'):
        """Initialize the chatbot with optimized settings"""
        print("🤖 Initializing Enhanced Medical Chatbot...")
        
        load_dotenv()
        
        print(f"🧠 Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name)
        
        print("🔌 Connecting to Pinecone...")
        PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
        if not PINECONE_API_KEY:
            raise ValueError("❌ PINECONE_API_KEY not found!")
        
        pc = Pinecone(api_key=PINECONE_API_KEY)
        self.index = pc.Index(index_name)
        
        stats = self.index.describe_index_stats()
        print(f"✅ Connected to index: {index_name}")
        print(f"📊 Total vectors: {stats['total_vector_count']}")
        print("=" * 60)
    
    def query(self, question, top_k=5, relevance_threshold=0.3):
        """
        Query with improved relevance filtering
        
        Args:
            question: Medical question
            top_k: Number of chunks to retrieve
            relevance_threshold: Minimum similarity score (0-1)
        """
        print(f"\n❓ Question: {question}")
        print("-" * 60)
        
        # Generate query embedding
        question_embedding = self.model.encode(question).tolist()
        
        # Search with more candidates
        results = self.index.query(
            vector=question_embedding,
            top_k=top_k,
            include_metadata=True
        )
        
        if not results['matches']:
            return "❌ No relevant information found."
        
        # Filter by relevance threshold
        relevant_matches = [m for m in results['matches'] if m['score'] >= relevance_threshold]
        
        if not relevant_matches:
            print(f"⚠️ No results above relevance threshold ({relevance_threshold})")
            print(f"   Best score found: {results['matches'][0]['score']:.3f}")
            print(f"   Showing top result anyway...\n")
            relevant_matches = results['matches'][:1]
        
        print(f"✅ Found {len(relevant_matches)} relevant chunks (score ≥ {relevance_threshold})\n")
        
        # Format answer
        answer = self._format_smart_answer(question, relevant_matches)
        
        return answer
    
    def _format_smart_answer(self, question, matches):
        """Smart formatting that prioritizes the best matches"""
        
        answer = "📚 **Answer:**\n\n"
        
        # Show top 3 most relevant chunks
        for i, match in enumerate(matches[:3], 1):
            score = match['score']
            text = match['metadata']['text']
            
            # Clean up the text
            text = self._clean_display_text(text)
            
            if i == 1:
                # Primary answer (highest relevance)
                answer += f"{text}\n\n"
            else:
                # Additional context
                answer += f"**Additional context (relevance: {score:.2f}):**\n{text}\n\n"
        
        # Add source information
        answer += "─" * 60 + "\n"
        answer += f"📎 **Sources:** {len(matches)} relevant chunks found\n"
        
        for i, match in enumerate(matches, 1):
            score = match['score']
            keywords = match['metadata'].get('keywords', 'N/A')
            chunk_id = match['metadata'].get('chunk_id', 'N/A')
            answer += f"   {i}. Score: {score:.3f} | Chunk: {chunk_id} | Keywords: {keywords}\n"
        
        return answer
    
    def _clean_display_text(self, text):
        """Clean text for better readability"""
        # Remove excessive whitespace
        text = re.sub(r'\s+', ' ', text)
        # Remove weird formatting artifacts
        text = re.sub(r'\s+([.,;:!?])', r'\1', text)
        return text.strip()
    
    def ask(self, question, show_details=False):
        """
        Simplified query method - just get the answer
        
        Args:
            question: Medical question
            show_details: Show detailed source info
        """
        question_embedding = self.model.encode(question).tolist()
        
        results = self.index.query(
            vector=question_embedding,
            top_k=5,
            include_metadata=True
        )
        
        if not results['matches']:
            return "No information found."
        
        # Get best match
        best_match = results['matches'][0]
        answer_text = best_match['metadata']['text']
        score = best_match['score']
        
        # Clean and format
        answer_text = self._clean_display_text(answer_text)
        
        if show_details:
            return f"[Relevance: {score:.2f}]\n\n{answer_text}"
        else:
            return answer_text
    
    def chat(self):
        """Interactive chat mode with better UX"""
        print("\n" + "=" * 60)
        print("🏥 MEDICAL CHATBOT - Interactive Mode")
        print("=" * 60)
        print("Ask medical questions naturally!")
        print("Commands: 'exit' to quit | 'help' for tips")
        print("=" * 60 + "\n")
        
        while True:
            try:
                question = input("💬 You: ").strip()
                
                if not question:
                    continue
                
                if question.lower() in ['quit', 'exit', 'q', 'bye']:
                    print("\n👋 Take care!")
                    break
                
                if question.lower() == 'help':
                    print("\n💡 Tips for better results:")
                    print("   - Be specific: 'What is plasma?' vs 'plasma'")
                    print("   - Ask complete questions")
                    print("   - Use medical terminology when known")
                    print("   - Try rephrasing if results aren't good\n")
                    continue
                
                # Get answer
                print()  # Blank line for readability
                answer = self.query(question, top_k=5, relevance_threshold=0.3)
                print(f"🤖 Bot:\n\n{answer}\n")
                print("─" * 60 + "\n")
                
            except KeyboardInterrupt:
                print("\n\n👋 Take care!")
                break
            except Exception as e:
                print(f"\n❌ Error: {str(e)}\n")


# ============================================
# CONVENIENT FUNCTIONS
# ============================================

def quick_ask(question, index_name="medical-knowledge-v2"):
    """Ultra-simple: Just ask and get an answer"""
    chatbot = MedicalChatbot(index_name=index_name)
    return chatbot.ask(question)

def detailed_query(question, index_name="medical-knowledge-v2"):
    """Get detailed answer with sources"""
    chatbot = MedicalChatbot(index_name=index_name)
    return chatbot.query(question, top_k=5, relevance_threshold=0.3)


# ============================================
# DEMO USAGE
# ============================================

def main():
    # Initialize
    chatbot = MedicalChatbot(index_name="medical-knowledge-v2")
    
    # Test queries with better relevance
    test_questions = [
        "What is plasma?",
        "What are red blood cells?",
        "Explain hemoglobin",
        "What is anemia?",
        "Define leukocytes"
    ]
    
    print("\n🧪 TESTING ENHANCED QUERY SYSTEM")
    print("=" * 60)
    
    for question in test_questions:
        print(f"\n📌 Testing: {question}")
        print("-" * 60)
        answer = chatbot.query(question, top_k=5, relevance_threshold=0.35)
        print(answer)
        print("\n" + "=" * 60)
        
        # Pause between queries
        input("\nPress Enter for next question...")
    
    print("\n\n💬 Switching to interactive mode...")
    print("(You can now ask your own questions!)\n")
    
    # Start interactive chat
    chatbot.chat()


if __name__ == "__main__":
    # Option 1: Run full demo
    # Option 2: Quick single query (uncomment to use)
    # print(quick_ask("What is plasma?"))
    
    # Option 3: Detailed query (uncomment to use)
    # print(detailed_query("What is plasma?"))

🤖 Initializing Enhanced Medical Chatbot...
🧠 Loading embedding model: all-MiniLM-L6-v2...
🔌 Connecting to Pinecone...
✅ Connected to index: medical-knowledge-v2
📊 Total vectors: 909

🧪 TESTING ENHANCED QUERY SYSTEM

📌 Testing: What is plasma?
------------------------------------------------------------

❓ Question: What is plasma?
------------------------------------------------------------
✅ Found 2 relevant chunks (score ≥ 0.35)

📚 **Answer:**

Above the RBClayer inacentrifuged sample isalayer ofwhite blood cells (WBCs) andplatelets, which together make upless than 1% ofthe sample of whole blood. Above this buffy layer isplasma (PLAZ-mă), theliquid part of blood. Plasma isapale, straw-colored fluid that constitutes 55% ofablood sample. Plasma ismostly water with dissolved proteins, including albumin, immunoglobulins, andclotting factors. Italso contains nutrients, electrolytes, andcellular wastes. Plasma suspends RBCs, WBCs, andplatelets, enabling them tocirculate throughout thebody 

In [1]:
import os
import re
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
from dotenv import load_dotenv
import time

class FastMedicalChatbot:
    """Optimized chatbot with CORRECTED relevance thresholds for all-MiniLM-L6-v2"""
    
    _instance = None
    _model = None
    _index = None
    _initialized = False
    
    # Medical domain keywords for query validation
    MEDICAL_KEYWORDS = {
        'blood', 'cell', 'cells', 'disease', 'diseases', 'symptom', 'symptoms',
        'treatment', 'treatments', 'diagnosis', 'diagnose', 'medicine', 'medication',
        'drug', 'drugs', 'therapy', 'patient', 'patients', 'doctor', 'hospital',
        'infection', 'infections', 'virus', 'viruses', 'bacteria', 'bacterial',
        'organ', 'organs', 'tissue', 'tissues', 'bone', 'bones', 'muscle', 'muscles',
        'heart', 'cardiac', 'lung', 'pulmonary', 'brain', 'neural', 'liver', 'hepatic',
        'kidney', 'renal', 'pain', 'ache', 'fever', 'temperature', 'cancer', 'tumor',
        'diabetes', 'diabetic', 'pressure', 'hypertension', 'medical', 'health',
        'anatomy', 'physiology', 'pathology', 'syndrome', 'condition', 'disorder',
        'immune', 'immunity', 'antibody', 'antigen', 'protein', 'enzyme', 'hormone',
        'plasma', 'serum', 'hemoglobin', 'anemia', 'leukemia', 'thrombosis',
        'what is', 'what are', 'explain', 'define', 'how does', 'why does'
    }
    
    def __new__(cls, index_name="medical-knowledge-v2"):
        if cls._instance is None:
            cls._instance = super(FastMedicalChatbot, cls).__new__(cls)
        return cls._instance
    
    def __init__(self, index_name="medical-knowledge-v2"):
        if not FastMedicalChatbot._initialized:
            print("⚡ Initializing Fast Medical Chatbot (one-time setup)...")
            start_time = time.time()
            
            load_dotenv()
            
            print("🧠 Loading embedding model: all-MiniLM-L6-v2...")
            FastMedicalChatbot._model = SentenceTransformer('all-MiniLM-L6-v2')
            
            print("🔌 Connecting to Pinecone...")
            PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
            if not PINECONE_API_KEY:
                raise ValueError("❌ PINECONE_API_KEY not found!")
            
            pc = Pinecone(api_key=PINECONE_API_KEY)
            FastMedicalChatbot._index = pc.Index(index_name)
            
            stats = FastMedicalChatbot._index.describe_index_stats()
            FastMedicalChatbot._initialized = True
            
            init_time = time.time() - start_time
            print(f"✅ Ready! (loaded in {init_time:.2f}s)")
            print(f"📊 Total vectors: {stats['total_vector_count']}")
            print("=" * 60)
    
    def _is_likely_medical_query(self, question):
        """Check if question appears to be medical-related"""
        question_lower = question.lower()
        return any(keyword in question_lower for keyword in self.MEDICAL_KEYWORDS)
    
    def _validate_query(self, question):
        """Validate and sanitize input query"""
        if not isinstance(question, str):
            return False, "Please provide a text question."
        
        question = question.strip()
        
        if not question:
            return False, "Please provide a valid question."
        
        if len(question) < 3:
            return False, "Question is too short. Please be more specific."
        
        return True, question
    
    def query(self, question, top_k=3, show_score=False, min_score=0.35, strict_mode=True):
        """
        Query with CORRECTED relevance filtering for all-MiniLM-L6-v2
        
        Args:
            question: User's question
            top_k: Number of results to retrieve
            show_score: Show relevance scores
            min_score: Minimum relevance threshold (ADJUSTED FOR THIS MODEL)
                      0.35-0.40: Balanced (RECOMMENDED for all-MiniLM-L6-v2)
                      0.45-0.50: Strict (only highly relevant)
                      0.25-0.35: Lenient (more results)
            strict_mode: If True, also check for medical keywords
        """
        start_time = time.time()
        
        # Validate input
        is_valid, result = self._validate_query(question)
        if not is_valid:
            return f"❌ {result}"
        
        question = result
        
        # Optional: Check if query seems medical-related
        if strict_mode and not self._is_likely_medical_query(question):
            return (
                "❌ Your question doesn't appear to be medical-related. "
                "I specialize in medical information. Please ask about:\n"
                "• Medical conditions, diseases, or symptoms\n"
                "• Anatomy and physiology\n"
                "• Medical terminology\n"
                "• Treatments and medications\n\n"
                "Or set strict_mode=False if you believe this is a medical query."
            )
        
        try:
            # Generate embedding
            question_embedding = self._model.encode(question).tolist()
            
            # Search Pinecone
            results = self._index.query(
                vector=question_embedding,
                top_k=top_k,
                include_metadata=True
            )
            
            if not results['matches']:
                return "❌ No information found in the database."
            
            # CRITICAL: Filter by relevance score (CORRECTED THRESHOLDS)
            relevant_matches = [
                match for match in results['matches'] 
                if match['score'] >= min_score
            ]
            
            if not relevant_matches:
                best_score = results['matches'][0]['score']
                return (
                    f"❌ No sufficiently relevant information found.\n"
                    f"(Best match score: {best_score:.3f}, threshold: {min_score:.3f})\n\n"
                    "💡 Your content might be in the database but below the relevance threshold.\n"
                    "Try:\n"
                    "• Lowering min_score to 0.25-0.30\n"
                    "• Rephrasing your question\n"
                    "• Using more specific medical terms\n"
                    "• Checking if the topic is covered in your PDF"
                )
            
            query_time = time.time() - start_time
            answer = self._format_answer(relevant_matches, show_score, query_time)
            
            return answer
            
        except Exception as e:
            return f"❌ Error processing query: {str(e)}"
    
    def _format_answer(self, matches, show_score, query_time):
        """Format answer with relevance information"""
        answer = ""
        
        # Best result
        best_match = matches[0]
        text = best_match['metadata']['text']
        text = re.sub(r'\s+', ' ', text).strip()
        score = best_match['score']
        
        # Add confidence indicator (ADJUSTED FOR ALL-MINILM-L6-V2)
        if score >= 0.60:
            confidence = "🟢 Excellent"
        elif score >= 0.45:
            confidence = "🟢 High"
        elif score >= 0.35:
            confidence = "🟡 Good"
        elif score >= 0.25:
            confidence = "🟠 Moderate"
        else:
            confidence = "🔴 Low"
        
        answer += f"**Confidence: {confidence} (Score: {score:.3f})**\n\n"
        answer += f"{text}\n"
        
        # Additional context
        if len(matches) > 1:
            answer += "\n" + "─" * 60 + "\n"
            answer += "📚 Additional context:\n\n"
            
            for i, match in enumerate(matches[1:], 2):
                if i > 3:  # Limit to 3 additional sources
                    break
                text = match['metadata']['text']
                text = re.sub(r'\s+', ' ', text).strip()
                score = match['score']
                
                # Truncate smartly
                if len(text) > 300:
                    last_period = text[:300].rfind('. ')
                    if last_period > 200:
                        text = text[:last_period + 1]
                    else:
                        text = text[:300] + "..."
                
                answer += f"[{i}] (relevance: {score:.3f})\n{text}\n\n"
        
        # Performance info
        if show_score:
            answer += "─" * 60 + "\n"
            answer += f"⚡ Query time: {query_time:.3f}s | Results: {len(matches)}\n"
        
        return answer
    
    def ask(self, question, min_score=0.35, strict_mode=True):
        """
        Simple query method with CORRECTED relevance filtering
        
        Args:
            question: User's question
            min_score: Minimum relevance threshold (default: 0.35 for all-MiniLM-L6-v2)
            strict_mode: Check for medical keywords (default: True)
        """
        # Validate
        is_valid, result = self._validate_query(question)
        if not is_valid:
            return f"Error: {result}"
        
        question = result
        
        # Check medical relevance
        if strict_mode and not self._is_likely_medical_query(question):
            return "This doesn't appear to be a medical question. Please ask about medical topics."
        
        try:
            question_embedding = self._model.encode(question).tolist()
            results = self._index.query(
                vector=question_embedding,
                top_k=1,
                include_metadata=True
            )
            
            if not results['matches']:
                return "No information found."
            
            match = results['matches'][0]
            score = match['score']
            
            # Check relevance (CORRECTED THRESHOLD)
            if score < min_score:
                return f"No sufficiently relevant information found (score: {score:.3f}, threshold: {min_score:.3f})"
            
            text = match['metadata']['text']
            cleaned_text = re.sub(r'\s+', ' ', text).strip()
            
            return f"[Score: {score:.3f}] {cleaned_text}"
            
        except Exception as e:
            return f"Error: {str(e)}"


# ==================================================================================================================================
#                                                        HELPER FUNCTIONS
# ==================================================================================================================================

_chatbot = None

def _get_chatbot():
    """Get or create chatbot instance (lazy initialization)"""
    global _chatbot
    if _chatbot is None:
        _chatbot = FastMedicalChatbot()
    return _chatbot

def ask(question, min_score=0.35, strict_mode=True):
    """
    Ultra-simple function - just ask!
    
    Args:
        question: Your medical question
        min_score: Minimum relevance score (0.35 recommended for all-MiniLM-L6-v2)
        strict_mode: Require medical keywords in question
    """
    start = time.time()
    chatbot = _get_chatbot()
    answer = chatbot.ask(question, min_score=min_score, strict_mode=strict_mode)
    elapsed = time.time() - start
    print(f"⚡ Answered in {elapsed:.3f}s\n")
    return answer

def query(question, top_k=3, min_score=0.35, strict_mode=True):
    """
    Detailed query with sources
    
    Args:
        question: Your medical question
        top_k: Number of results to retrieve
        min_score: Minimum relevance score (0.35 recommended for all-MiniLM-L6-v2)
        strict_mode: Require medical keywords in question
    """
    chatbot = _get_chatbot()
    return chatbot.query(
        question, 
        top_k=top_k, 
        show_score=True, 
        min_score=min_score,
        strict_mode=strict_mode
    )

def batch_ask(questions, min_score=0.35, strict_mode=True):
    """
    Ask multiple questions efficiently
    
    Args:
        questions: List of questions
        min_score: Minimum relevance score
        strict_mode: Require medical keywords
    """
    print(f"🚀 Processing {len(questions)} questions...\n")
    total_start = time.time()
    
    chatbot = _get_chatbot()
    results = {}
    
    for i, question in enumerate(questions, 1):
        start = time.time()
        answer = chatbot.ask(question, min_score=min_score, strict_mode=strict_mode)
        elapsed = time.time() - start
        
        print(f"{i}. {question}")
        print(f"   ⚡ {elapsed:.3f}s")
        print(f"   📝 {answer[:100]}...\n")
        
        results[question] = answer
    
    total_time = time.time() - total_start
    avg_time = total_time / len(questions) if questions else 0
    print(f"✅ Total time: {total_time:.2f}s | Average: {avg_time:.2f}s per query")
    
    return results

def test_relevance_threshold(test_queries=None, min_score=0.35):
    """
    Test the relevance threshold with known queries
    
    Args:
        test_queries: List of (question, should_pass) tuples
        min_score: Threshold to test (default: 0.35 for all-MiniLM-L6-v2)
    """
    if test_queries is None:
        test_queries = [
            ("What is plasma?", True),
            ("Explain hemoglobin", True),
            ("What is diabetes?", True),
            ("What is anemia?", True),
            ("Define leukocytes", True),
            ("What is Farhan?", False),
            ("Tell me about Ronaldo", False),
            ("What is love?", False),
            ("Hello how are you?", False),
        ]
    
    print(f"🧪 Testing Relevance Threshold: {min_score}")
    print("=" * 70 + "\n")
    
    chatbot = _get_chatbot()
    passed = 0
    failed = 0
    
    for query_text, should_pass in test_queries:
        result = chatbot.query(query_text, min_score=min_score, strict_mode=True, show_score=False)
        has_error = "❌" in result
        
        # Check if result matches expectation
        if should_pass and not has_error:
            status = "✅ PASS"
            passed += 1
        elif not should_pass and has_error:
            status = "✅ PASS"
            passed += 1
        else:
            status = "❌ FAIL"
            failed += 1
        
        print(f"{status} | {query_text}")
        print(f"       Expected: {'ACCEPT' if should_pass else 'REJECT'}")
        print(f"       Result: {result[:80]}...")
        print()
    
    print("=" * 70)
    print(f"Results: {passed} passed, {failed} failed out of {len(test_queries)} tests")
    print(f"Accuracy: {(passed/len(test_queries)*100):.1f}%")
    
    return passed, failed

def diagnose_scores():
    """Diagnose what scores you're actually getting"""
    print("🔍 SCORE DIAGNOSTIC TOOL")
    print("=" * 70)
    print("Testing various queries to see actual similarity scores...\n")
    
    chatbot = _get_chatbot()
    
    test_queries = [
        "What is plasma?",
        "What is anemia?",
        "Explain hemoglobin",
        "What is diabetes?",
        "What is Farhan?",
        "Tell me about football",
    ]
    
    for question in test_queries:
        embedding = chatbot._model.encode(question).tolist()
        results = chatbot._index.query(
            vector=embedding,
            top_k=1,
            include_metadata=True
        )
        
        if results['matches']:
            score = results['matches'][0]['score']
            text = results['matches'][0]['metadata']['text'][:100]
            print(f"❓ {question}")
            print(f"   Score: {score:.3f}")
            print(f"   Match: {text}...")
            print()
    
    print("=" * 70)
    print("💡 Recommended thresholds for all-MiniLM-L6-v2:")
    print("   • 0.35-0.40: Balanced (catches most relevant)")
    print("   • 0.25-0.35: Lenient (more results)")
    print("   • 0.45-0.50: Strict (only best matches)")

print("✅ Enhanced chatbot loaded with CORRECTED thresholds!")
print("💡 Use min_score=0.35 (recommended for all-MiniLM-L6-v2)")
print("📊 Run diagnose_scores() to see what scores you're actually getting")

c:\Users\HP\miniconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Enhanced chatbot loaded with CORRECTED thresholds!
💡 Use min_score=0.35 (recommended for all-MiniLM-L6-v2)
📊 Run diagnose_scores() to see what scores you're actually getting


In [6]:
answer = ask("What is bilirubin?")
print(answer)

⚡ Answered in 2.249s

[Score: 0.511] Figure 12.11 Structures Related tothe Liver Bilirubin (bil-ĭ-ROO-bin) isayellowish pigment produced bythe breakdown ofred blood cells that isfiltered outofthe blood bythe liver. Ifaperson’sliver 32. “Cenveo – Drawing Liver anatomy andvascularisation – English labels” by Cenveo islicensed under CC BY 4.0 33. This work isaderivative ofAnatomy andPhysiology byOpenStax licensed under CC BY 4.0. Access forfree at 12.4 Anatomy ofthe Digestive System | 691


In [3]:
# Require medical keywords (recommended for production)
ask("What is plasma?", strict_mode=True)  # DEFAULT

# Allow any question (for testing)
ask("What is plasma?", strict_mode=False)

⚡ Answered in 0.397s

⚡ Answered in 0.347s



'[Score: 0.399] Above the RBClayer inacentrifuged sample isalayer ofwhite blood cells (WBCs) andplatelets, which together make upless than 1% ofthe sample of whole blood. Above this buffy layer isplasma (PLAZ-mă), theliquid part of blood. Plasma isapale, straw-colored fluid that constitutes 55% ofablood sample. Plasma ismostly water with dissolved proteins, including albumin, immunoglobulins, andclotting factors. Italso contains nutrients, electrolytes, andcellular wastes. Plasma suspends RBCs, WBCs, andplatelets, enabling them tocirculate throughout thebody within thecardiovascular system. 2 See Figure 10.1 3 foranillustration ofthe composition ofblood inablood sample. examinations (3rd ed., Chapter 151). Butterworths. 2. This work isaderivative ofAnatomy & Physiology byOpenStax andislicensed under CC BY 4.0. Access forfree at anatomy-and-physiology-2e 3. “1913_ABO_Blood_Groups.jpg” byAlan Sved islicensed under CC BY-SA 4.0. 10.4 Anatomy ofthe Hematology System | 561'

In [ ]:
# Get more context
query("What is diabetes?", top_k=5)


# Just the best match
query("What is diabetes?", top_k=1)

'**Confidence: 🟢 Excellent (Score: 0.627)**\n\nFigure 17.4 Dilute Urine Seen in Polyuria DIABETES MELLITUS Diabetes mellitus (dī-ă-BĒ-tēz MĔL-ĭ-tŭs) (DM) refers todisorders that cause hyperglycemia (high levels ofglucose inthe blood). Over time, hyperglycemia damages theblood vessels inthe heart, kidneys, feet, andeyes. 9. “31388976010_b0750ceed4_k.jpg” byJohn Campbell islicensed inthe Public Domain. 10. National Institute ofDiabetes andDigestive andKidney Disease. (2021, September). Diabetes insipidus. National Institutes ofHealth. Diseases andDisorders ofthe Endocrine System\n────────────────────────────────────────────────────────────\n⚡ Query time: 0.406s | Results: 1\n'